### JAI SHREE RAM

Stage 2: Predicting Length of Stay

In [151]:
import pandas as pd
import numpy as np

In [152]:
#import admisson data
df = pd.read_csv('admission_data.csv')

In [153]:
df


,MRD No.,AGE,GENDER,SMOKING,ALCOHOL,HTN,CAD,PRIOR CMP,HB,TLC,...,VT,PSVT,CONGENITAL,NEURO CARDIOGENIC SYNCOPE,ORTHOSTATIC,CARDIOGENIC SHOCK,SHOCK,PULMONARY EMBOLISM,CHEST INFECTION,TARGET
0,506,50,M,0,0,1,1,0,13.1,10.5,...,0,0,0,0,0,0,0,0,0,4
1,798,71,M,0,0,0,1,0,12.9,8.5,...,0,0,0,0,0,0,0,0,0,2
2,798,71,M,0,0,0,1,0,12.9,8.5,...,0,0,0,0,0,0,0,0,0,2
3,798,72,M,0,0,0,1,0,11.4,6.6,...,0,0,0,0,0,0,0,0,0,6
4,989,71,M,0,0,1,1,0,10.7,14.1,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15752,4888286,68,M,0,0,0,1,0,12.2,6.7,...,0,0,0,0,0,0,0,0,0,5
15753,4888926,65,F,0,0,1,1,0,11.3,11.3,...,0,0,0,0,0,0,0,0,0,2
15754,5711587,60,F,0,0,0,0,1,10.1,9.8,...,0,0,0,0,0,0,0,0,0,11
15755,6408503,68,F,0,0,0,1,0,9.8,17.3,...,0,0,0,0,0,0,1,0,0,11


In [154]:
#find correlation between all the variables
corr_matrix = df.corr()
#remove correlated variables above 0.9
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column] > 0.80)]

# Drop features 
df.drop(to_drop, axis=1, inplace=True)

C:\Users\purja\AppData\Local\Temp\ipykernel_17596\4158322129.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr()


In [155]:
#list unique values in chest infection
df['CHEST INFECTION'].unique()
#drop rows with '\\' in chest infection
df = df[df['CHEST INFECTION'] != '\\']

In [163]:
df.dropna(inplace=True)
df = df[df['MRD No.'] != 'NILL']
df = df[(df != 'EMPTY').all(1)]
df['GENDER'] = np.where(df['GENDER'] == 'M', 1, 0)
#convert MRD No. to int
df['MRD No.'] = df['MRD No.'].astype(int)
#convert HB and TLC to float 
df['HB'] = df['HB'].astype(float)
df['TLC'] = df['TLC'].astype(float)
#convert Platelets, glucose and chest infection to int
# df['PLATELETS'] = df['PLATELETS'].astype(float)
# df['GLUCOSE'] = df['GLUCOSE'].astype(float)
df['CHEST INFECTION'] = df['CHEST INFECTION'].astype(int)


In [164]:
df.dtypes

MRD No.                        int32
AGE                            int64
GENDER                         int32
SMOKING                        int64
ALCOHOL                        int64
HTN                            int64
CAD                            int64
PRIOR CMP                      int64
HB                           float64
TLC                          float64
PLATELETS                    float64
GLUCOSE                      float64
RAISED CARDIAC ENZYMES         int64
SEVERE ANAEMIA                 int64
ANAEMIA                        int64
STABLE ANGINA                  int64
ACS                            int64
STEMI                          int64
ATYPICAL CHEST PAIN            int64
HEART FAILURE                  int64
VALVULAR                       int64
CHB                            int64
SSS                            int64
AF                             int64
VT                             int64
PSVT                           int64
CONGENITAL                     int64
N

In [165]:
#regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

#split data into train and test and drop Target and MRD No. variable from X
X = df.drop(['TARGET','MRD No.'], axis=1)
y = df['TARGET']


In [166]:
#fit the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
reg = LinearRegression().fit(X_train, y_train)

#predict the model
y_pred = reg.predict(X_test)

#calculate accuracy
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

#accuracy_score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred.round())


0.13208191126279864